# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
def Jacobi_iteration(A, b, eps = 1e-12, n_iter = 100):
    
    diag_1d = np.diag(A)

    B = -A.copy()

    np.fill_diagonal(B, 0)
    invD = np.diag(1./diag_1d)    
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(n)
    for a in range(n_iter):
      
        temp = BB @ x + c
        if np.linalg.norm(temp - x) < eps: 
            x = temp
            break
        x = temp

    return x

Checking correctness

In [ ]:
x = Jacobi_iteration(A, b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

Making diagonal smaller, check for convergence

In [ ]:
for i in range(1, 25):

    A_i = A + np.diagflat([-i]*n)
    norm_b = np.linalg.norm(np.diag(1./np.diag(A_i))@(-A_i.copy()+np.diag(np.diag(A_i))))
    convergence = np.linalg.norm(Jacobi_iteration(A_i, b)-np.linalg.solve(A_i, b))

    print(str(i)+".",
          "Norm of B", norm_b,
          "Convergence:", convergence)
    

1. Norm of B 0.38959181027260875 Convergence: 2.005305120107153e-17
2. Norm of B 0.4185783948614869 Convergence: 1.6502682545529e-17
3. Norm of B 0.4522284025473819 Convergence: 3.622208959480119e-17
4. Norm of B 0.4917667095178099 Convergence: 2.72080745804227e-17
5. Norm of B 0.5388887887486234 Convergence: 2.5332308935446513e-17
6. Norm of B 0.5960110344093966 Convergence: 2.715271730114205e-17
7. Norm of B 0.6667001660296402 Convergence: 3.1986720415755645e-17
8. Norm of B 0.7564517359241753 Convergence: 4.2668555758937507e-17
9. Norm of B 0.8742017351588476 Convergence: 1.306888029984837e-14
10. Norm of B 1.0355299928250665 Convergence: 2.726494964669312e-07
11. Norm of B 1.2702850939751231 Convergence: 178.83641240483422
12. Norm of B 1.6439565658213244 Convergence: 22252505496170.793
13. Norm of B 2.334809111760855 Convergence: 2.2347420503806264e+28
14. Norm of B 4.080768845910033 Convergence: 6.13577109020183e+51
15. Norm of B 30.715327603064885 Convergence: 7.280323737239156e

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
def seidel_iteration(A, b, eps = 1e-12, n_iter = 50):
    
    x = np.zeros(len(b))
    
    for i in range(n_iter):
      x_copy = x.copy()

      for j in range(b.shape[0]):
        x_copy[j] = (b[j]-np.dot(A[j][:j], x[:j])- np.dot(A[j][j+1:], x[j+1:]))/A[j,j]
      
      if np.linalg.norm(x_copy - x, ord=np.inf) / np.linalg.norm(x_copy, ord=np.inf) < eps:
        x = x_copy
        break

      x = x_copy
    return x

Checking correctness

In [ ]:
x = seidel_iteration(A,b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

Making diagonal smaller, check for convergence

In [ ]:
for i in range(1, 25):

    A_i = A + np.diagflat([-i]*n)
    norm_b = np.linalg.norm(np.diag(1./np.diag(A_i))@(-A_i.copy()+np.diag(np.diag(A_i))))
    convergence = np.linalg.norm(seidel_iteration(A_i, b)-np.linalg.solve(A_i, b))

    print(str(i)+".",
          "Norm of B", norm_b,
          "Convergence:", convergence)

1. Norm of B 0.38959181027260875 Convergence: 1.441843549426575e-14
2. Norm of B 0.4185783948614869 Convergence: 3.377066387387329e-14
3. Norm of B 0.4522284025473819 Convergence: 3.984221259521332e-14
4. Norm of B 0.4917667095178099 Convergence: 3.174930488165913e-14
5. Norm of B 0.5388887887486234 Convergence: 5.48869792195361e-14
6. Norm of B 0.5960110344093966 Convergence: 5.620681427479863e-14
7. Norm of B 0.6667001660296402 Convergence: 7.886279447627353e-14
8. Norm of B 0.7564517359241753 Convergence: 7.643869710323461e-12
9. Norm of B 0.8742017351588476 Convergence: 1.1267351603270573e-08
10. Norm of B 1.0355299928250665 Convergence: 5.682933757898043e-05
11. Norm of B 1.2702850939751231 Convergence: 1.6249187890365402
12. Norm of B 1.6439565658213244 Convergence: 649343.4829966454
13. Norm of B 2.334809111760855 Convergence: 23791844542337.92
14. Norm of B 4.080768845910033 Convergence: 1.4925298432550307e+25
15. Norm of B 30.715327603064885 Convergence: 2.508140171395458e+60


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minimum_res_scheme(A, b, eps = 1e-12, n_iter = 100):
    
    x = np.zeros(len(b))
    x_copy = np.zeros(len(b))

    for i in range(n_iter):
      residual = A @ x - b
      tau = (residual @ A @ residual) / np.dot( A @ residual, A @ residual )
      x_copy = x- tau*residual

      if np.linalg.norm(x_copy - x) < eps:
        x = x_copy 
        break
      x = x_copy

    return x

Checking correctness

In [ ]:
x = minimum_res_scheme(A,b)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

Making diagonal smaller, check for convergence

In [ ]:
for i in range(1, 25):

    A_i = A + np.diagflat([-i]*n)
    norm_b = np.linalg.norm(np.diag(1./np.diag(A_i))@(-A_i.copy()+np.diag(np.diag(A_i))))
    convergence = np.linalg.norm(minimum_res_scheme(A_i, b)-np.linalg.solve(A_i, b))

    print(str(i)+".",
          "Norm of B", norm_b,
          "Convergence:", convergence)

1. Norm of B 0.38959181027260875 Convergence: 6.673028063636746e-14
2. Norm of B 0.4185783948614869 Convergence: 1.767697481147239e-13
3. Norm of B 0.4522284025473819 Convergence: 7.368052439887803e-14
4. Norm of B 0.4917667095178099 Convergence: 5.5194981071777194e-14
5. Norm of B 0.5388887887486234 Convergence: 2.1579490351292104e-13
6. Norm of B 0.5960110344093966 Convergence: 1.9335756405058914e-13
7. Norm of B 0.6667001660296402 Convergence: 3.1197922640425305e-13
8. Norm of B 0.7564517359241753 Convergence: 2.0319648079364555e-13
9. Norm of B 0.8742017351588476 Convergence: 2.944061117089313e-13
10. Norm of B 1.0355299928250665 Convergence: 4.79355622288367e-13
11. Norm of B 1.2702850939751231 Convergence: 3.0887364779793784e-13
12. Norm of B 1.6439565658213244 Convergence: 6.004620874100477e-13
13. Norm of B 2.334809111760855 Convergence: 1.5917901613494577e-12
14. Norm of B 4.080768845910033 Convergence: 1.863422634148316e-08
15. Norm of B 30.715327603064885 Convergence: 1.7352